
# Clasificación de Fractales con Redes Neuronales Convolucionales (CNN)

En este notebook, aprenderás cómo clasificar fractales utilizando aprendizaje profundo. Usaremos un modelo basado en CNN con **transfer learning**.



## Paso 1: Preparación y análisis del dataset

Cargaremos los datos y visualizaremos algunas imágenes de fractales.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import os

# Cargar datos
train_df = pd.read_csv('train.csv')
validation_df = pd.read_csv('validation.csv')

# Mostrar ejemplos del dataset
train_df.head()



### Visualización de ejemplos de fractales


In [ ]:

from PIL import Image

def show_images(df, folder='train', num_images=4):
    fig, axes = plt.subplots(1, num_images, figsize=(15,5))
    for i in range(num_images):
        img_path = os.path.join(folder, df.iloc[i]['relative_path'])
        image = Image.open(img_path)
        axes[i].imshow(image)
        axes[i].set_title(df.iloc[i]['fractal'])
        axes[i].axis('off')
    plt.show()

show_images(train_df)



## Paso 2: Preprocesamiento de datos y Data Augmentation

Utilizaremos técnicas de aumento de datos para mejorar la robustez del modelo.


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1,
                             rotation_range=15, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=True)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='train',
    x_col='relative_path',
    y_col='fractal',
    target_size=(224, 224),
    batch_size=32,
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='train',
    x_col='relative_path',
    y_col='fractal',
    target_size=(224, 224),
    batch_size=32,
    subset='validation'
)



## Paso 3: Definición y entrenamiento del modelo CNN con Transfer Learning

Utilizaremos EfficientNetB0 preentrenado en ImageNet.


In [ ]:

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(train_df['fractal'].nunique(), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



### Entrenamiento del modelo


In [ ]:

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)



## Paso 4: Evaluación del modelo

Visualizaremos la precisión del modelo durante el entrenamiento.


In [ ]:

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()
